In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import re
import math
import json
import nltk
from sklearn.decomposition import PCA
from scipy.spatial import distance
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer, PorterStemmer 

In [3]:
docs_json = json.load(open("cran_docs.json", 'r'))[:]
doc_ids, docs = [item["id"] for item in docs_json],[item["body"] for item in docs_json]

In [4]:
queries_json = json.load(open("cran_queries.json", 'r'))[:]
query_ids, queries = [item["query number"] for item in queries_json],[item["query"] for item in queries_json]

In [5]:
qrels = json.load(open("cran_qrels.json", 'r'))[:]

In [6]:
tokenize = RegexpTokenizer(r'\b\w{1,}\b')
stop_words = set(stopwords.words("english"))

In [7]:
class LemmaTokenizer(object):
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`', '(', ')', '+', '-', '--', '*', '/', '']
    def __init__(self):
        self.wnl = PorterStemmer()
    def __call__(self, articles):
        return [self.wnl.stem(re.sub(r'[0-9]', "", t)) for t in tokenize.tokenize(articles) if re.sub(r'[0-9]', "", t) not in self.ignore_tokens]

In [8]:
tokenizer=LemmaTokenizer()
token_stop = tokenizer(' '.join(stop_words))

In [9]:
t = 4287
d = 1390
p = 10
k = 300

In [10]:
sample_tfidf = TfidfVectorizer(lowercase = True, stop_words = token_stop, tokenizer = tokenizer) #, min_df = 0.1)
sample_sparse = sample_tfidf.fit_transform(docs[:d])
tfidf = sample_sparse.toarray()

c:\python37\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [11]:
tfidf.shape

(1390, 4287)

In [12]:
A = np.transpose(tfidf)
#A = Ahat[:,:d]
D = sample_tfidf.transform(docs[d:])
D = np.transpose(D.toarray())

In [13]:
D.shape

(4287, 10)

In [14]:
u, s, vh = np.linalg.svd(A, full_matrices = False)
Uk = u[:,:k]
Sk = np.diag(s[:k])
Vk = np.transpose(vh[:k,:])

In [15]:
#Dcap = np.matmul(np.ones((t,t)) - np.matmul(Uk, np.transpose(Uk)), D)

In [16]:
#Dcap.shape

In [17]:
#Qd, Rd = np.linalg.qr(Dcap, mode='reduced')

In [18]:
#Rd.shape

In [19]:
#a = np.append(Sk, np.matmul(np.transpose(Uk), D), axis = 1)
#b = np.append(np.zeros((p, k)), Rd, axis = 1)
Acap = np.append(Sk, np.matmul(np.transpose(Uk), D), axis = 1)

In [20]:
Acap.shape

(300, 310)

In [21]:
Ucap, Scap, Vcapt = np.linalg.svd(Acap, full_matrices = False)
Vcap = np.transpose(Vcapt)
#Ukcap = u[:,:k]
#Upcap = u[:,k:]
#Skcap = np.diag(s[:k])
#Spcap = np.diag(s[k:])
#Vkcap = np.transpose(vh)[:,:k]
#Vpcap = np.transpose(vh)[:,k:]

In [22]:
T = np.matmul(Uk, Ucap)

In [23]:
S = np.diag(Scap)

In [24]:
a = np.append(Vk, np.zeros((d, p)), axis = 1)
b = np.append(np.zeros((p, k)), np.ones((p,p)), axis = 1)
V = np.matmul(np.append(a, b, axis = 0), Vcap)

In [25]:
Dt = np.transpose(V)

In [26]:
S.shape

(300, 300)

In [27]:
sample_sparse_q = sample_tfidf.transform(queries)
tfidf_q = sample_sparse_q.toarray()

In [28]:
def LSI(n_components, u, s, vh, tfidf_q):
    T = u[:,:n_components]
    S = np.diag(s[:n_components])
    Dt = vh[:n_components,:]
    doc_vectors = np.dot(np.transpose(Dt), S)
    query_vectors = np.dot(tfidf_q, T)
    return doc_vectors, query_vectors

In [29]:
def rank(doc_vectors, query_vectors, docIDs):
    doc_IDs_ordered = list()
    for q_vector in query_vectors:
        retrieved_docs = dict()
        key = 0
        for doc_vector in doc_vectors:
            cosine = 1 - distance.cosine(doc_vector, q_vector)
            retrieved_docs[docIDs[key]] = cosine
            key += 1
        doc_IDs_ordered.append(sorted(retrieved_docs,reverse=True,key = lambda x: retrieved_docs[x]))
    return doc_IDs_ordered

In [30]:
class Evaluation():

	def queryPrecision(self, query_doc_IDs_ordered, query_id, true_doc_IDs, k):
		"""
		Computation of precision of the Information Retrieval System
		at a given value of k for a single query

		Parameters
		----------
		arg1 : list
			A list of integers denoting the IDs of documents in
			their predicted order of relevance to a query
		arg2 : int
			The ID of the query in question
		arg3 : list
			The list of IDs of documents relevant to the query (ground truth)
		arg4 : int
			The k value

		Returns
		-------
		float
			The precision value as a number between 0 and 1
		"""

		precision = -1
		retrieved_docs = query_doc_IDs_ordered[:k]
		n_relevant_docs = 0
		
		for doc in retrieved_docs:
			if doc in true_doc_IDs:
				n_relevant_docs+=1
		
		precision = n_relevant_docs/k

		return precision


	def meanPrecision(self, doc_IDs_ordered, query_ids, qrels, k):
		"""
		Computation of precision of the Information Retrieval System
		at a given value of k, averaged over all the queries

		Parameters
		----------
		arg1 : list
			A list of lists of integers where the ith sub-list is a list of IDs
			of documents in their predicted order of relevance to the ith query
		arg2 : list
			A list of IDs of the queries for which the documents are ordered
		arg3 : list
			A list of dictionaries containing document-relevance
			judgements - Refer cran_qrels.json for the structure of each
			dictionary
		arg4 : int
			The k value

		Returns
		-------
		float
			The mean precision value as a number between 0 and 1
		"""

		meanPrecision = -1
		total_Precision = 0
		count = 0
		for i in query_ids:
			true_doc_IDs = []
			flag = 0
			for dic in qrels:
				if(int(dic["query_num"]) == i):
					flag = 1
					true_doc_IDs.append(int(dic["id"]))
				elif(flag == 1):
					break     
			total_Precision = total_Precision + self.queryPrecision(doc_IDs_ordered[count] , i , true_doc_IDs, k)
			count = count+1
		if count == 0:
	  		count = 1
		meanPrecision = total_Precision/count

		return meanPrecision

	
	def queryRecall(self, query_doc_IDs_ordered, query_id, true_doc_IDs, k):
		"""
		Computation of recall of the Information Retrieval System
		at a given value of k for a single query

		Parameters
		----------
		arg1 : list
			A list of integers denoting the IDs of documents in
			their predicted order of relevance to a query
		arg2 : int
			The ID of the query in question
		arg3 : list
			The list of IDs of documents relevant to the query (ground truth)
		arg4 : int
			The k value

		Returns
		-------
		float
			The recall value as a number between 0 and 1
		"""

		recall = -1
		retrieved_docs = query_doc_IDs_ordered[:k]
		n_relevant_docs = 0
		for doc in retrieved_docs:
			if doc in  true_doc_IDs:
				n_relevant_docs += 1
		recall = n_relevant_docs/len(true_doc_IDs)

		return recall


	def meanRecall(self, doc_IDs_ordered, query_ids, qrels, k):
		"""
		Computation of recall of the Information Retrieval System
		at a given value of k, averaged over all the queries

		Parameters
		----------
		arg1 : list
			A list of lists of integers where the ith sub-list is a list of IDs
			of documents in their predicted order of relevance to the ith query
		arg2 : list
			A list of IDs of the queries for which the documents are ordered
		arg3 : list
			A list of dictionaries containing document-relevance
			judgements - Refer cran_qrels.json for the structure of each
			dictionary
		arg4 : int
			The k value

		Returns
		-------
		float
			The mean recall value as a number between 0 and 1
		"""

		meanRecall = -1
		total_Recall = 0
		count = 0
		for i in query_ids:
			true_doc_IDs = []
			flag = 0
			for dic in qrels:
				if(int(dic["query_num"]) == i):
					flag = 1
					true_doc_IDs.append(int(dic["id"]))
				elif(flag == 1):
					break
			total_Recall = total_Recall + self.queryRecall(doc_IDs_ordered[count] , i , true_doc_IDs, k)
			count = count+1
		if count == 0:
	  		count = 1
		meanRecall = total_Recall/count

		return meanRecall


	def queryFscore(self, query_doc_IDs_ordered, query_id, true_doc_IDs, k):
		"""
		Computation of fscore of the Information Retrieval System
		at a given value of k for a single query

		Parameters
		----------
		arg1 : list
			A list of integers denoting the IDs of documents in
			their predicted order of relevance to a query
		arg2 : int
			The ID of the query in question
		arg3 : list
			The list of IDs of documents relevant to the query (ground truth)
		arg4 : int
			The k value

		Returns
		-------
		float
			The fscore value as a number between 0 and 1
		"""

		fscore = -1
		precision = self.queryPrecision(query_doc_IDs_ordered , query_id , true_doc_IDs, k)
		recall = self.queryRecall(query_doc_IDs_ordered , query_id , true_doc_IDs, k)
		fscore = (2*precision*recall)/(precision + recall + 0.00001)

		return fscore


	def meanFscore(self, doc_IDs_ordered, query_ids, qrels, k):
		"""
		Computation of fscore of the Information Retrieval System
		at a given value of k, averaged over all the queries

		Parameters
		----------
		arg1 : list
			A list of lists of integers where the ith sub-list is a list of IDs
			of documents in their predicted order of relevance to the ith query
		arg2 : list
			A list of IDs of the queries for which the documents are ordered
		arg3 : list
			A list of dictionaries containing document-relevance
			judgements - Refer cran_qrels.json for the structure of each
			dictionary
		arg4 : int
			The k value
		
		Returns
		-------
		float
			The mean fscore value as a number between 0 and 1
		"""

		meanFscore = -1
		total_Fscore = 0
		count = 0
		for i in query_ids:
			true_doc_IDs = []
			flag = 0
			for dic in qrels:
				if(int(dic["query_num"]) == i):
					flag = 1
					true_doc_IDs.append(int(dic["id"]))
				elif(flag == 1):
					break
			total_Fscore = total_Fscore + self.queryFscore(doc_IDs_ordered[count] , i , true_doc_IDs, k)
			count = count+1
		if count == 0:
	  		count = 1
		meanFscore = total_Fscore/count

		return meanFscore
	

	def queryNDCG(self, query_doc_IDs_ordered, query_id, qrels, k):
		"""
		Computation of nDCG of the Information Retrieval System
		at given value of k for a single query

		Parameters
		----------
		arg1 : list
			A list of integers denoting the IDs of documents in
			their predicted order of relevance to a query
		arg2 : int
			The ID of the query in question
		arg3 : list
			A list of dictionaries containing document-relevance
			judgements - Refer cran_qrels.json for the structure of each
			dictionary
		arg4 : int
			The k value

		Returns
		-------
		float
			The nDCG value as a number between 0 and 1
		"""

		nDCG = -1
		DCG = 0
		iDCG = 0
		retrieved_docs = query_doc_IDs_ordered[:k]
		true_doc_IDs = dict()
		flag = 0
		
		for dic in qrels:
			if(int(dic["query_num"]) == query_id):
				flag = 1
				true_doc_IDs[int(dic["id"])] = 5 - dic['position']
			elif(flag == 1):
				break
		ideal_order = sorted(query_doc_IDs_ordered, key = lambda x: true_doc_IDs[x] if x in true_doc_IDs else 0, reverse = True)[:k]
		for i in range(1,k+1):
			if retrieved_docs[i-1] in true_doc_IDs:
				DCG += true_doc_IDs[retrieved_docs[i-1]]/math.log2(i+1)
			if ideal_order[i-1] in true_doc_IDs:
				iDCG += true_doc_IDs[ideal_order[i-1]]/math.log2(i+1)

		if iDCG == 0:
			iDCG = 1	
		nDCG = DCG/iDCG

		return nDCG


	def meanNDCG(self, doc_IDs_ordered, query_ids, qrels, k):
		"""
		Computation of nDCG of the Information Retrieval System
		at a given value of k, averaged over all the queries

		Parameters
		----------
		arg1 : list
			A list of lists of integers where the ith sub-list is a list of IDs
			of documents in their predicted order of relevance to the ith query
		arg2 : list
			A list of IDs of the queries for which the documents are ordered
		arg3 : list
			A list of dictionaries containing document-relevance
			judgements - Refer cran_qrels.json for the structure of each
			dictionary
		arg4 : int
			The k value

		Returns
		-------
		float
			The mean nDCG value as a number between 0 and 1
		"""

		meanNDCG = -1
		total_NDCG = 0
		count = 0
		for i in query_ids:
			total_NDCG = total_NDCG + self.queryNDCG(doc_IDs_ordered[count] , i , qrels, k)
			count = count+1
		if count == 0:
	  		meanNDCG = 0
		else:
			meanNDCG = total_NDCG/count

		return meanNDCG


	def queryAveragePrecision(self, query_doc_IDs_ordered, query_id, true_doc_IDs, k):
		"""
		Computation of average precision of the Information Retrieval System
		at a given value of k for a single query (the average of precision@i
		values for i such that the ith document is truly relevant)

		Parameters
		----------
		arg1 : list
			A list of integers denoting the IDs of documents in
			their predicted order of relevance to a query
		arg2 : int
			The ID of the query in question
		arg3 : list
			The list of documents relevant to the query (ground truth)
		arg4 : int
			The k value

		Returns
		-------
		float
			The average precision value as a number between 0 and 1
		"""

		avgPrecision = -1
		retrieved_docs = query_doc_IDs_ordered[:k]
		Total_Precision = 0
		count = 0
		for i in range(len(retrieved_docs)):
			doc = retrieved_docs[i]
			if doc in true_doc_IDs:
				count += 1
				Total_Precision += self.queryPrecision(query_doc_IDs_ordered, query_id, true_doc_IDs, i+1) 
		if count == 0:
			avgPrecision = 0
		else:      
			avgPrecision = Total_Precision/count

		return avgPrecision


	def meanAveragePrecision(self, doc_IDs_ordered, query_ids, qrels, k):
		"""
		Computation of MAP of the Information Retrieval System
		at given value of k, averaged over all the queries

		Parameters
		----------
		arg1 : list
			A list of lists of integers where the ith sub-list is a list of IDs
			of documents in their predicted order of relevance to the ith query
		arg2 : list
			A list of IDs of the queries
		arg3 : list
			A list of dictionaries containing document-relevance
			judgements - Refer cran_qrels.json for the structure of each
			dictionary
		arg4 : int
			The k value

		Returns
		-------
		float
			The MAP value as a number between 0 and 1
		"""

		meanAveragePrecision = -1
		total_AveragePrecision = 0
		count = 0
		for i in query_ids:
			true_doc_IDs = []
			flag = 0
			for dic in qrels:
				if(int(dic["query_num"]) == i):
					flag = 1
					true_doc_IDs.append(int(dic["id"]))
				elif(flag == 1):
					break
			total_AveragePrecision = total_AveragePrecision + self.queryAveragePrecision(doc_IDs_ordered[count] , i , true_doc_IDs, k)
			count = count+1
		if count == 0:
	  		count = 1
		meanAveragePrecision = total_AveragePrecision/count

		return meanAveragePrecision

In [31]:
evaluator = Evaluation()

In [32]:
n_components = 300 # Parameter to tune
doc_vectors, query_vectors = LSI(n_components, T, S, Dt, tfidf_q)
doc_IDs_ordered = rank(doc_vectors, query_vectors, doc_ids)

In [33]:
# Calculate precision, recall, f-score, MAP and nDCG for k = 1 to 10
precisions, recalls, fscores, MAPs, nDCGs = [], [], [], [], []
for k in range(1, 11):
    precision = evaluator.meanPrecision(doc_IDs_ordered, query_ids, qrels, k)
    precisions.append(precision)
    recall = evaluator.meanRecall(doc_IDs_ordered, query_ids, qrels, k)
    recalls.append(recall)
    fscore = evaluator.meanFscore(doc_IDs_ordered, query_ids, qrels, k)
    fscores.append(fscore)
    print("Precision, Recall and F-score @ " +  str(k) + " : " + str(precision) + ", " + str(recall) + ", " + str(fscore))
    MAP = evaluator.meanAveragePrecision(doc_IDs_ordered, query_ids, qrels, k)
    MAPs.append(MAP)
    nDCG = evaluator.meanNDCG(doc_IDs_ordered, query_ids, qrels, k)
    nDCGs.append(nDCG)
    print("MAP, nDCG @ " +  str(k) + " : " + str(MAP) + ", " + str(nDCG))

Precision, Recall and F-score @ 1 : 0.008888888888888889, 0.0011851851851851852, 0.002037019483176399
MAP, nDCG @ 1 : 0.008888888888888889, 0.0077777777777777776
Precision, Recall and F-score @ 2 : 0.006666666666666667, 0.0014814814814814816, 0.0023155655363588925
MAP, nDCG @ 2 : 0.008888888888888889, 0.005847423866488529
Precision, Recall and F-score @ 3 : 0.005925925925925926, 0.0019753086419753087, 0.0028394563280804224
MAP, nDCG @ 3 : 0.010370370370370368, 0.005256966580840124
Precision, Recall and F-score @ 4 : 0.0044444444444444444, 0.0019753086419753087, 0.0026070318640519555
MAP, nDCG @ 4 : 0.010370370370370368, 0.004521649822037544
Precision, Recall and F-score @ 5 : 0.0044444444444444444, 0.0023793490460157127, 0.002968175576160028
MAP, nDCG @ 5 : 0.011259259259259259, 0.004612647814091636
Precision, Recall and F-score @ 6 : 0.0074074074074074086, 0.00467678494345161, 0.00542926910591536
MAP, nDCG @ 6 : 0.014962962962962961, 0.00606655254353258
Precision, Recall and F-score @

In [128]:
 a = np.reshape(np.array([4.0,5.0,1.0,7.0]), (-1, 1))

In [129]:
a.shape

(4, 1)